In [268]:
import numpy as np
import read_data
import math

In [269]:
print (train_data.shape)
print (valid_data.shape)
print (test_data.shape)
type(train_data.shape)

(27000, 15)
(3000, 15)
(7000, 15)


tuple

# Node Declaration

In [270]:
# Node Declaration

class Node:
    
    def __init__(self):
        self.splitAttribute=-1
        self.pred=-1
        self.positives=0
        self.negatives=0
        self.children={}
        self.median=-1
        self.isNumeric=False
        self.ChildrenCorrectlyClassified=0

In [271]:
def getMedian(data,splitAttribute):
        return np.median(data[:,splitAttribute])

# Growing Tree

In [272]:
def divideData(validData,validLabel,splitAttribute,isNumeric):
    distributeData={}
    distributeLabel={}
    instances=len(validData)
    
    
    if(isNumeric):
        numpyData=np.asarray(validData,dtype=np.float32)
        median=getMedian(numpyData,splitAttribute)
        for i in range(instances):
            key=-1
            if(valid_data[i][splitAttribute]<=median):
                key=0
            else:
                key=1
            if key in distributeData:
                distributeData[key].append(validData[i])
                distributeLabel[key].append(validLabel[i])
            
            else:
                distributeData[key]=[validData[i]]
                distributeLabel[key]=[validLabel[i]]

    else:
        for i in range(instances):
            key=validData[i][splitAttribute]
            
            if key in distributeData:
                distributeData[key].append(validData[i])
                distributeLabel[key].append(validLabel[i])
            
            else:
                distributeData[key]=[validData[i]]
                distributeLabel[key]=[validLabel[i]]
        
    return (distributeData,distributeLabel)
    

In [273]:
def growTree(data,label,hasBeenSplit,isNumeric):
    
    global nodes
    nodes+=1
#     print (nodes,len(data))
#     if(len(data)==3):
#         print (data)
#     get Best Splitting Attribute
    splitAttribute=getSplitAttribute(data,label,hasBeenSplit,isNumeric)
    
    instances=len(data)
    positives=0.0
    negatives=0.0
    
    for i in range(instances):
        if(label[i]==1):
            positives+=1
        else:
            negatives+=1
#     Declaring and Initialising Root
    root=Node()
    root.splitAttribute=splitAttribute
    root.positives=positives
    root.negatives=negatives
    if(positives>=negatives):
        root.pred=1
    else:
        root.pred=0
        
    if(splitAttribute==-1):
        return root
    
    hasBeenSplit[splitAttribute]=True

    if(isNumeric[splitAttribute]):
        root.isNumeric=True
        
    (distributeData,distributeLabel)=divideData(data,label,splitAttribute,root.isNumeric)
    
    if(len(distributeLabel)==1):
        root.splitAttribute=-1
        hasBeenSplit[splitAttribute]=False
        return root
    
    for key,value in distributeData.items():
        root.children[key]=growTree(value,distributeLabel[key],hasBeenSplit,isNumeric)
#         root.children[key].parent=root
        
    hasBeenSplit[splitAttribute]=False
    return root

# Selecting Best Attribute

In [274]:
def getSplitAttribute(data,label,hasBeenSplit,isNumeric):
    attributes=len(hasBeenSplit)
    
    curEntropy=Hy(label)
    
    bestAttribute=-1
    conditionalEntropy=1e9
    instances=len(data)
    numpyData=np.asarray(data,dtype=np.float32)
        
    for i in range(attributes):
#         if(isNumeric[i]):
#             print("HI")
#             pass
        if(hasBeenSplit[i] and not isNumeric[i]):
            continue
        counterTable={}
        median=-1
        if(isNumeric[i]):
            median=getMedian(numpyData,i)
#         print(i)
        
        for j in range(instances):
            key=-1
            if(not isNumeric[i]):
                key=data[j][i]
            else:
                if(data[j][i]<=median):
                    key=0
                else:
                    key=1
            
            if key in counterTable:
                counterTable[key][int(label[j])]+=1
            else:
                counterTable[key]=[0,0]
#                 print (counterTable)
                counterTable[key][int(label[j])]+=1
        
        entropy=0
        for key,value in counterTable.items():
            prob=(value[0]+value[1])/float(instances)
            entropy+=prob*getEntropy(value[1],value[0])
        
#         print(entropy)
        if(entropy<conditionalEntropy):
            conditionalEntropy=entropy
            bestAttribute=i
    
#     print(curEntropy)
#     print(conditionalEntropy)
    
    informationGain=curEntropy-conditionalEntropy
    
#     print(informationGain)
    
    if(informationGain<=0):
        return -1
    
    return bestAttribute
            
                
        
        

In [275]:
def Hy(y):
    instances=len(y)
    positives=0
    negatives=0
    
    for i in range(instances):
        if(y[i]==0):
            negatives+=1
        else:
            positives+=1

    return getEntropy(positives,negatives)

In [276]:
def getEntropy(positives,negatives):
    p1=positives/float(positives+negatives)
    p0=negatives/float(positives+negatives)
    
    entropy=0
    
    if(negatives!=0):
        entropy+=(-p0*math.log(p0))
    
    if(positives!=0):
        entropy+=(-p1*math.log(p1))
    
    return entropy

# Prediction & Accuracy

In [277]:
def getPrediction(root,instance):
    if(root.splitAttribute==-1):
        return root.pred
    
    key=-1
    
    if(root.isNumeric):
        if(instance[root.splitAttribute]<=root.median):
            key=0
        else:
            key=1
    else:
        key=instance[root.splitAttribute]
    
    if key in root.children:
        return getPrediction(root.children[key],instance)
    else:
        return root.pred



In [278]:
def getAccuracy(root,data,label):
    
    instance=len(data)
    correct=0
    
    for i in range(instance):
        if(label[i]==getPrediction(root,data[i])):
            correct+=1
    
    return correct/float(instance)

In [279]:
getEntropy(1,4)

0.5004024235381879

In [280]:
# test_data

In [281]:
# b=np.array([1,2,3])
# b.shape
# type((b.tolist())[0])

# Pruning

In [282]:
def prune(root,originalRoot,validData,validLabel):
    
    bestAccuracy=0
    bestParent=root
    bestKey=-1
    
    temp=dict(root.children)
    
    for key,value in temp.items():
#         print(key)
        root.children.pop(key, None)
        accuracy=getAccuracy(originalRoot,validData,validLabel)
        if(accuracy>bestAccuracy):
            bestAccuracy=accuracy
            bestKey=key
        root.children[key]=value
    
    for key,value in root.children.items():
        tupleA=prune(value,originalRoot,validData,validLabel)
        if(tupleA[0]>bestAccuracy):
            bestAccuracy=tupleA[0]
            bestParent=tupleA[1]
            bestKey=tupleA[2]
    
    return (bestAccuracy,bestParent,bestKey)
        

In [283]:
# def efficientPrune(root,originalRoot,validData,validLabel,firstTime):
    
#     bestAccuracy=0
#     bestParent=root
#     bestKey=-1
    
#     temp=dict(root.children)
    
#     if(not firstTime and root.recalculate==False):
#         return root.prunedTuple

#     for key,value in temp.items():
#         root.children.pop(key, None)
#         accuracy=getAccuracy(originalRoot,validData,validLabel)
#         if(accuracy>bestAccuracy):
#             bestAccuracy=accuracy
#             bestKey=key
#         root.children[key]=value
    
#     for key,value in root.children.items():
#         tupleA=efficientPrune(value,originalRoot,validData,validLabel,firstTime)
#         if(tupleA[0]>bestAccuracy):
#             bestAccuracy=tupleA[0]
#             bestParent=tupleA[1]
#             bestKey=tupleA[2]
    
#     root.recalculate=False
#     root.prunedTuple=(bestAccuracy,bestParent,bestKey)
#     return (bestAccuracy,bestParent,bestKey)
        

In [284]:
# def updateTillParent(root):
    
#     while root is not None:
#         root.recalculate=True
#         root=root.parent
        

# Run Slow Pruning

In [285]:
# formerBest=getAccuracy(root,validX,validY)
# while True:
#     print ("Hi")
#     (bestAccuracy,pruneNodeParent,prunedKey)=prune(root,root,validX,validY)
#     print(formerBest," ",bestAccuracy)
#     if(bestAccuracy>formerBest):
#         pruneNodeParent.children.pop(prunedKey, None)
#         formerBest=bestAccuracy
#     else:
#         break

In [286]:
# formerBest=getAccuracy(root,validX,validY)
# firstTime=True
# while True:
#     print ("Hi")
#     (bestAccuracy,pruneNodeParent,prunedKey)=efficientPrune(root,root,validX,validY,firstTime)
#     firstTime=False
#     print(formerBest," ",bestAccuracy)
#     if(bestAccuracy>formerBest):
#         pruneNodeParent.children.pop(prunedKey, None)
#         formerBest=bestAccuracy
#         updateTillParent(pruneNodeParent)
#     else:
#         break

In [287]:
# https://leemendelowitz.github.io/blog/how-does-python-find-packages.html

# Efficient Pruning

In [288]:
def newprune(root,validData,validLabel):
    
    bestIncrease=0
    bestNode=root

    if(root.splitAttribute==-1):
        return (bestIncrease,bestNode)
    
    pruneCorrect=0
    instances=len(validData)
    for i in range(instances):
        if(validLabel[i]==root.pred):
            pruneCorrect+=1
    
#     print (pruneCorrect,root.ChildrenCorrectlyClassified)
    if(bestIncrease<(pruneCorrect-root.ChildrenCorrectlyClassified)):
        bestIncrease=(pruneCorrect-root.ChildrenCorrectlyClassified)
        
    (distributeData,distributeLabel)=divideData(validData,validLabel,root.splitAttribute,root.isNumeric)
    
    for key,value in root.children.items():
        if key not in distributeData:
            continue
        tupleA=newprune(value,distributeData[key],distributeLabel[key])
        if(tupleA[0]>bestIncrease):
            bestIncrease=tupleA[0]
            bestNode=tupleA[1]
    
    return (bestIncrease,bestNode)

In [289]:
def prePrune(root,validData,validLabel):
    
    instances=len(validData)
    correctlyDone=0
    if(root.splitAttribute==-1):
        for i in range(instances):
            if(validLabel[i]==root.pred):
                correctlyDone+=1
    
    else:
        (distributeData,distributeLabel)=divideData(validData,validLabel,root.splitAttribute,root.isNumeric)
        
        for key,value in distributeData.items():
            if key in root.children:
                correctlyDone+=prePrune(root.children[key],value,distributeLabel[key])
#                 root.children[key]
            else:
                temp=len(value)
                for i in range(temp):
                    if(distributeLabel[key][i]==root.pred):
                        correctlyDone+=1
    
    root.ChildrenCorrectlyClassified=correctlyDone
#     print (correctlyDone)
    return correctlyDone

# Testing Code Training and Pruning (Part a and b)

In [290]:
train_data = read_data.preprocess("Dataset/DT/dtree_data/train.csv")
valid_data = read_data.preprocess("Dataset/DT/dtree_data/valid.csv")
test_data = read_data.preprocess("Dataset/DT/dtree_data/test.csv")

In [238]:
# train_data = read_data.preprocess("Dataset/DT/dtree_data/train.csv",False)
# valid_data = read_data.preprocess("Dataset/DT/dtree_data/valid.csv",False)
# test_data = read_data.preprocess("Dataset/DT/dtree_data/test.csv",False)

In [291]:
def getLists(train_data):
    (instances,attributes)=train_data.shape
    trainLabel=train_data[:,0]
    trainData=train_data[:,1:attributes]
    trainDataList=trainData.tolist()
    trainLabelList=trainLabel.tolist()
    return(trainDataList,trainLabelList)


In [292]:
(trainX,trainY)=getLists(train_data)
(validX,validY)=getLists(valid_data)
(testX,testY)  =getLists(test_data)

In [293]:
hasBeenSplit=[]
isNumeric=[]
attributes=train_data.shape[1]
for i in range(attributes-1):
    hasBeenSplit.append(False)
    isNumeric.append(False)

# isNumeric[0]=True
# isNumeric[2]=True
# isNumeric[4]=True
# isNumeric[10]=True
# isNumeric[11]=True
# isNumeric[12]=True

In [296]:
# print (validX[0])
# print(validX[1])

In [297]:
nodes=0
root=growTree(trainX,trainY,hasBeenSplit,isNumeric)

In [298]:
print (nodes)
print (getAccuracy(root,trainX,trainY))
print (getAccuracy(root,testX,testY))
print (getAccuracy(root,validX,validY))

7902
0.8893333333333333
0.8024285714285714
0.7916666666666666


In [299]:
while True:
    val=prePrune(root,validX,validY)
    print ("PrePruned")
    (bestIncrease,bestNode)=newprune(root,validX,validY)
    print(bestIncrease)
    if(bestIncrease>0):
        bestNode.splitAttribute=-1
    else:
        break

PrePruned
38
PrePruned
9
PrePruned
5
PrePruned
5
PrePruned
4
PrePruned
3
PrePruned
3
PrePruned
3
PrePruned
3
PrePruned
2
PrePruned
2
PrePruned
2
PrePruned
2
PrePruned
2
PrePruned
2
PrePruned
2
PrePruned
2
PrePruned
2
PrePruned
2
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
1
PrePruned
0


In [300]:
print (nodes)
print (getAccuracy(root,trainX,trainY))
print (getAccuracy(root,testX,testY))
print (getAccuracy(root,validX,validY))

7902
0.8507777777777777
0.8232857142857143
0.835
